# Création du fichier de test complet

## Packages

In [40]:
import pandas as pd             # Traitement des tableaux de données 

## Chargement des données

Dans un premier temps, l'objectif est de récupérer le fichier complet puis de ne conserver que les variables présentes au sein du fichier d'entraînement (et bien entendu la variable 'Num_Acc' qui servira de repère). 4 fichiers composent le test set. Ainsi, il faudra itérer pour récupérer l'ensemble des données :

In [2]:
files = ['USAGERS', 'VEHICULES', 'CARACTERISTIQUES', 'LIEUX',]
data_path = 'Data/TEST/TEST/'

full_data = pd.DataFrame()

for file_name in files: 
    path_to_file = data_path + file_name + '.csv'
    
    data = pd.read_csv(path_to_file, encoding="latin1", sep=",", low_memory=False)
    
    if file_name == 'USAGERS':
        full_data = pd.concat([full_data, data], axis=1)
    else:
        if file_name == 'VEHICULES':
            full_data = pd.merge(full_data, data, on=['Num_Acc', 'num_veh'], how='left')
        else:
            full_data = pd.merge(full_data, data, on=['Num_Acc'], how='left')

In [3]:
initial_row = full_data.shape[0]
initial_col = full_data.shape[1]

print(f"Le nombre d'observations initial est de : {initial_row}")
print(f"Le nombre de variables initial est de : {initial_col}")

Le nombre d'observations initial est de : 142422
Le nombre de variables initial est de : 58


On récupère en tout **'*142 422*'** observations. Cela comprend tous les accidentés pour tous les accidents. Ainsi, lors de la soumission, il faudra fournir la moyenne (par défaut) pour chaque accident.

In [4]:
full_data.to_csv('Filtered_Data/TEST/Full_Test_Data_Initial.csv', index=False)

Désormais, nous disposons d'un **"*.csv*"** de grandes amplitudes. La première étape avant de pouvoir utiliser ce jeu de données en tant que jeu de test, est de conserver uniquement les variables présentent au sein du train set.

## Sélection des variables et Harmonisation des données

### Fonctions utiles

In [41]:
def delete_columns(list_col: list[str], data_frame: pd.DataFrame) -> None:
    """
    Delete the selected columns from the data frame
    :param list_col: list of columns in the dataframe to delete
    :param data_frame: dataframe where the deletion is applied
    """
    for col in list_col:
        if col in data_frame.columns:
            data_frame.drop(labels=[col], axis=1, inplace=True)
            
def process_secu(value: int) -> int:
    """
    Process the conversion from a 2 digits integer to 1 digits integer
    :param value: to be converted
    :return: only the digit associated with the security element (or 8 if nothing used)
    """
    value_str = str(value)
    if len(value_str) >= 2 and value_str[1] == '1':
        return int(value_str[0])
    else:
        return pd.NA
    
def convert_to_int(time_str: str) -> int:
    """
    Convert the string time in an integer value to allow intervals implementation
    :param time_str: 'XX:XX', 'X:XX', 'XXX', 'X', X, XXX, ...
    :return: an integer of the time according to the time format XX(hours)XX(minutes)
    """
    if isinstance(time_str, str):
        if ':' in time_str:
            hours, minutes = map(int, time_str.split(':'))
            return hours * 100 + minutes
        else:
            return int(time_str)
    else:
        return time_str
    
def generate_intervals(start, end, step) -> list[int]:
    """
    Generate the intervals from the start to the end value by step
    :param start: first value in the interval
    :param end: last value in the interval
    :param step: range between two categories
    :return: list of intervals
    """
    list_cat = []
    for i in range(start, end, step):
        list_cat.append((i, i + step))
    return list_cat

def map_to_simple_value(value: int, list_cat: list[int]) -> int:
    """
    Function to map values to simple values based on intervals
    :param list_cat: list of the intervals
    :param value: value to map
    :return: mapped value
    """
    for i, interval in enumerate(list_cat):
        if interval[0] <= value < interval[1]:
            return simple_values[i]
    return None  

def clean_dep(dep: str) -> str:
    """
    Format the string of the department
    :param dep: French department
    :return: formatted string of the department
    """
    dep = str(dep)
    dep = dep.lstrip('0')
    if len(dep) == 3 and dep.endswith('0'):
        dep = dep[:-1]
    if dep in {'201', '202'}:
        dep = {'201': '2A', '202': '2B'}[dep]
    return dep
    
def convert_float_to_int(data_frame: pd.DataFrame, col_name: str) -> None:
    """
    Convert the column type in the related dataframe from float to int
    :param data_frame: dataframe where the column must be
    :param col_name: column to parse
    """
    data_frame[col_name] = data_frame[col_name].astype(int)

In [107]:
test_data = pd.read_csv('Filtered_Data/TEST/Full_Test_Data_Initial.csv', sep=',', low_memory=False)
train_data = pd.read_csv('Filtered_Data/TRAIN/Filtered_Train_Data_V4_to_V4.csv', sep=',', low_memory=False)

#### Usagers

Deux nuances sont apportées, on conserve la variable **'*Num_Acc*'** et on supprime le traitement d'une colonne inexistante **'*grav*'**. 

In [108]:
# Suppression des colonnes 
col_to_drop_usagers = ['id_vehicule_x', 'id_vehicule_y', 'num_veh', 'an_nais', 'locp', 'etatp', 'secu2', 'secu3', 'actp', 'id_usager']
delete_columns(col_to_drop_usagers, test_data)

# test_data['sexe']
test_data = test_data.replace({'sexe' : {2: 0, -1 : pd.NA}})

# test_data['trajet']
test_data = test_data.replace({'trajet': {-1: pd.NA, 0: pd.NA}})

# test_data['catu']
test_data.loc[test_data['catu'] == 4, 'catv'] = 99
test_data['catu'] = test_data['catu'].replace({4: 3})

# test_data['place']
test_data.loc[test_data['catu'] == 3, 'place'] = 10
test_data['place'] = test_data['place'].replace({-1: pd.NA}).fillna(pd.NA)

# test_data['secu', 'secu1']
test_data['secu'] = test_data['secu'].apply(process_secu)
test_data['secu'] = test_data['secu'].fillna(test_data['secu1'].replace([-1], pd.NA))
delete_columns(['secu1'], test_data)

Nous nous penchons sur les changements des dimensions entre les deux jeux de données (initial/post-usagers) :

In [109]:
post_usagers_row = test_data.shape[0]
post_usagers_col = test_data.shape[1]

print(f"Le nombre d'observations initial est de : {post_usagers_row} (différence de : {initial_row - post_usagers_row} lignes).")
print(f"Le nombre de variables initial est de : {post_usagers_col} (différence de : {initial_col - post_usagers_col} variables).")

Le nombre d'observations initial est de : 142422 (différence de : 0 lignes).
Le nombre de variables initial est de : 47 (différence de : 11 variables).


#### Caractéristiques

In [110]:
# Suppression des colonnes 
col_to_drop_usagers = ['com', 'adr', 'gps', 'lat', 'long']
delete_columns(col_to_drop_usagers, test_data)

# test_data['an']
test_data.loc[(test_data['an'] >= 12) & (test_data['an'] <= 18), 'an'] += 2000

# test_data['hrmn']
test_data['hrmn'] = test_data['hrmn'].apply(convert_to_int)
convert_float_to_int(test_data, 'hrmn')
intervals = generate_intervals(0, 2400, 100)
simple_values = list(range(len(intervals)))
test_data['hrmn'] = test_data['hrmn'].apply(map_to_simple_value, list_cat=intervals)

# test_data['lum']
test_data['lum'] = test_data['lum'].replace({-1 : pd.NA})

# test_data['agg']
test_data['agg'] = test_data['agg'].replace({2: 0})

# test_data['dep']
test_data['dep'] = test_data['dep'].apply(clean_dep)
mask = ~(test_data['dep'].isin(map(str, range(1, 96))) | test_data['dep'].isin(['971', '972', '974', '976']))
test_data.loc[mask, 'dep'] = pd.NA

# test_data['int']
test_data['int'] = test_data['int'].replace([-1, 0], pd.NA)

Nous nous penchons sur les changements des dimensions entre les deux jeux de données (post-usagers/post-caractéristiques) :

In [111]:
post_caract_row = test_data.shape[0]
post_caract_col = test_data.shape[1]

print(f"Le nombre d'observations initial est de : {post_caract_row} (différence de : {post_usagers_row - post_caract_row} lignes).")
print(f"Le nombre de variables initial est de : {post_caract_col} (différence de : {post_usagers_col - post_caract_col} variables).")

Le nombre d'observations initial est de : 142422 (différence de : 0 lignes).
Le nombre de variables initial est de : 42 (différence de : 5 variables).


#### Lieux

In [112]:
# Suppression des colonnes 
col_to_drop_usagers = ['voie', 'v1', 'v2', 'vosp', 'pr', 'pr1', 'lartpc', 'larrout', 'vma', 'env1', 'circ', 'prof']
delete_columns(col_to_drop_usagers, test_data)

# test_data['nbv']
replace_dict = {'0.0': pd.NA, '1.0': '1', '2.0': '2', '3.0': '3', '4.0': '4', '5.0': '5', '6.0': '6', '7.0': '7', '8.0': '8', '9.0': '9', '10.0': '10', '11.0': '11', '12.0' :'12', '13.0' : '13', '#ERREUR' : pd.NA, -1: pd.NA, '-1.0': pd.NA, ' -1' : pd.NA}
test_data['nbv'] = test_data['nbv'].replace(replace_dict)

# test_data['plan']
test_data['plan'] = test_data['plan'].replace([-1, 0], pd.NA)

# test_data['surf']
test_data['surf'] = test_data['surf'].replace([-1, 0], pd.NA)

# test_data['infra']
test_data['infra'] = test_data['infra'].replace({-1 : pd.NA})

# test_data['situ']
test_data['situ'] = test_data['situ'].replace([-1, 0], pd.NA)

Nous nous penchons sur les changements des dimensions entre les deux jeux de données (post-caractéristiques/post-lieux) :

In [113]:
post_lieux_row = test_data.shape[0]
post_lieux_col = test_data.shape[1]

print(f"Le nombre d'observations initial est de : {post_lieux_row} (différence de : {post_caract_row - post_lieux_row} lignes).")
print(f"Le nombre de variables initial est de : {post_lieux_col} (différence de : {post_caract_col - post_lieux_col} variables).")

Le nombre d'observations initial est de : 142422 (différence de : 0 lignes).
Le nombre de variables initial est de : 30 (différence de : 12 variables).


#### Véhicules

In [121]:
# Suppression des colonnes 
col_to_drop_usagers = ['senc', 'motor', 'occutc']
delete_columns(col_to_drop_usagers, test_data)

# test_data['catv']
test_data['catv'] = test_data['catv'].replace({-1: 0})

# test_data['obs']
test_data['obs'] = test_data['obs'].replace({-1: pd.NA})

# test_data['obsm']
test_data['obsm'] = test_data['obsm'].replace({-1: pd.NA})

# test_data['choc']
test_data['choc'] = test_data['choc'].replace({-1: pd.NA})

# test_data['manv']
test_data['manv'] = test_data['manv'].replace([-1, 0], pd.NA)

Nous nous penchons sur les changements des dimensions entre les deux jeux de données (post-lieux/post-véhicules) :

In [122]:
post_vehicules_row = test_data.shape[0]
post_vehicules_col = test_data.shape[1]

print(f"Le nombre d'observations initial est de : {post_vehicules_row} (différence de : {post_lieux_row - post_vehicules_row} lignes).")
print(f"Le nombre de variables initial est de : {post_vehicules_col} (différence de : {post_lieux_col - post_vehicules_col} variables).")

Le nombre d'observations initial est de : 142422 (différence de : 0 lignes).
Le nombre de variables initial est de : 27 (différence de : 3 variables).


In [123]:
test_data.to_csv('Filtered_Data/TEST/Full_Test_Data_Reduced.csv', index=False)

## Traitement des valeurs manquantes

In [46]:
test_data = pd.read_csv('Filtered_Data/TEST/Full_Test_Data_Reduced.csv', sep=',', low_memory=False)

### Parsing des variables 

Dans l'ensemble toutes les variables sont du style catégorielle. En effet, il est plus pertinent d'assigner ces valeurs numériques à des catégories que de réaliser une moyenne. Il serait faux de dire qu'un individu se trouve à la place 4, car 1 voisin vaut 0 et deux voisins valent 6. Il vaudrait mieux lui assigner la valeur 6 ainsi. L'important sera donc d'assigner un nombre impair au nombre de voisins.

In [47]:
for column in test_data.columns:
    test_data[column] = test_data[column].astype('category')

### Remplacement des variables manquantes

On remarque que seule le trajet manque de manière importante avec **27%** de valeurs manquantes. Ainsi, on décide de supprimer cette variable et de combler les autres variables par les valeurs les plus fréquentes.

In [48]:
# On récupère chaque colonne contenant une variable vide
col_with_missing = test_data.columns[test_data.isnull().any()].tolist()

# Pour chacune, on affiche son nom et son nombre de valeurs manquantes
for col in col_with_missing:
    missing_count = test_data[col].isnull().sum()
    print(f"Variable '{col}' has {missing_count} missing values, the ratio is {missing_count / test_data.shape[0] * 100}.")

Variable 'place' has 1 missing values, the ratio is 0.0007021387145244414.
Variable 'sexe' has 559 missing values, the ratio is 0.3924955414191628.
Variable 'trajet' has 38459 missing values, the ratio is 27.003552821895493.
Variable 'obs' has 59 missing values, the ratio is 0.04142618415694205.
Variable 'obsm' has 63 missing values, the ratio is 0.04423473901503981.
Variable 'choc' has 37 missing values, the ratio is 0.025979132437404337.
Variable 'manv' has 10955 missing values, the ratio is 7.691929617615257.
Variable 'lum' has 2 missing values, the ratio is 0.0014042774290488828.
Variable 'int' has 25 missing values, the ratio is 0.017553467863111034.
Variable 'atm' has 5 missing values, the ratio is 0.003510693572622207.
Variable 'dep' has 3019 missing values, the ratio is 2.1197567791492884.
Variable 'nbv' has 921 missing values, the ratio is 0.6466697560770106.
Variable 'plan' has 7299 missing values, the ratio is 5.124910477313898.
Variable 'surf' has 3341 missing values, the r

In [49]:
# Remplacement par la valeur la plus fréquente
for col in col_with_missing:
    most_frequent_value = test_data[col].mode()[0]
    test_data[col].fillna(most_frequent_value, inplace=True)
    
# Suppression de test_data['trajet']
delete_columns(['trajet'], test_data)
    
test_data.isna().sum().sum()

C:\Users\lucas\AppData\Local\Temp\ipykernel_5252\1526997875.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test_data[col].fillna(most_frequent_value, inplace=True)


0

In [50]:
test_data.to_csv('Filtered_Data/TEST/Full_Test_Data_Reduced_Mode.csv', index=False)

Désormais, nous pouvons nous pencher sur le feature engineering et appliquer les modèles sur ce jeu de données.